# Decision Trees

Decision Trees are powerful, versitile MLA's that can perform both classification & regression tasks, and event multioutput tasks. They can be used to fit complex datasets.

### Advantages: 

- Simple to understand and to interpret. Trees can be visualized.
- Requires little data preparation. Other techniques often require data normalization, dummy variables need to be created and blank values to be removed. Some tree and algorithm combinations support missing values.
- The cost of using the tree (i.e., predicting data) is logarithmic in the number of data points used to train the tree.
- Able to handle both numerical and categorical data. However, the scikit-learn implementation does not support categorical variables for now. Other techniques are usually specialized in analyzing datasets that have only one type of variable. See algorithms for more information.
- Able to handle multi-output problems.
- Uses a white box model. If a given situation is observable in a model, the explanation for the condition is easily explained by boolean logic. By contrast, in a black box model (e.g., in an artificial neural network), results may be more difficult to interpret.
- Possible to validate a model using statistical tests. That makes it possible to account for the reliability of the model.
- Performs well even if its assumptions are somewhat violated by the true model from which the data were generated.

### Disadvantages: 

- Decision-tree learners can create over-complex trees that do not generalize the data well. This is called overfitting. Mechanisms such as pruning, setting the minimum number of samples required at a leaf node or setting the maximum depth of the tree are necessary to avoid this problem.
- Decision trees can be unstable because small variations in the data might result in a completely different tree being generated. This problem is mitigated by using decision trees within an ensemble.
- Predictions of decision trees are neither smooth nor continuous, but piecewise constant approximations as seen in the above figure. Therefore, they are not good at extrapolation.
- The problem of learning an optimal decision tree is known to be NP-complete under several aspects of optimality and even for simple concepts. Consequently, practical decision-tree learning algorithms are based on heuristic algorithms such as the greedy algorithm where locally optimal decisions are made at each node. Such algorithms cannot guarantee to return the globally optimal decision tree. This can be mitigated by training multiple trees in an ensemble learner, where the features and samples are randomly sampled with replacement.
- There are concepts that are hard to learn because decision trees do not express them easily, such as XOR, parity or multiplexer problems.
- Decision tree learners create biased trees if some classes dominate. It is therefore recommended to balance the dataset prior to fitting with the decision tree.



## Training & Visualizing a Decision Tree

In [3]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

iris = load_iris()
X = iris.data[:, 2:] # Petal length & width
y = iris.target

tree_clf = DecisionTreeClassifier(max_depth=2)
tree_clf.fit(X, y)

DecisionTreeClassifier(max_depth=2)

You can visualize the trained DTree by first using the `export_graphviz` to output a graph definition file:

In [9]:
from sklearn.tree import export_graphviz

export_graphviz(
    tree_clf,
    out_file='iris_tree.dot',
    feature_names=iris.feature_names[2:],
    class_names=iris.target_names,
    rounded=True,
    filled=True
)

![Decision Tree](iris_tree.png)

## Making Predictions

Suppose you had an iris flower and you wanted to classifiy it - you would start at the root node, which asks if the petal lenght is greather than 2.45cm.

If it is, than you would move to the left child node (which is called a `leaf node` since it doesn't have any children nodes), so it does not ask any questions - it outputs a predicted class of `setosa`.

If the petal length is less than 2.45cm, than you would move down to right child node which asks another question - is the petal width smaller than 1.75cm?

If it is, you would move to the left leaf node which classifies the flower as `versicolor`. If it isn't, than you would move to the right leaf node which classifies the flower as a `virginica`.

The `sample` attributes counts how many training instances that node applies to,
The `value` attributes tells you how many training instances of each class this node applies to,
The `gini` attribute tells you how "pure" the node is by if all the training instances it applies to belong the same class,

## Estimating Class Probabilities

A decision tree can also estimate eht probability that an instance belogs to a particular class. First it traverses the tree to find the leaf node for this instance, and then it returns the ratio of training instances `k` in this node. 

For example, suppose you found a flower whose petals are tcm long and 1.5cm wide. The corresponding leaf node is the depth-2 left node, so the Dtree should output the following probabilities:

- 0% for Iris Setosa (0/54)
- 90.7% for Iris Versicolor (49/54)
- 9.3% fpr Iris Virginicia (5/54)

And if you ask it to predict the class, it should output `Iris Versicolor` because it has the highest probability:

In [14]:
tree_clf.predict_proba([[5, 1.5]])

array([[0.        , 0.90740741, 0.09259259]])

In [15]:
tree_clf.predict([[5, 1.5]])

array([1])

## The CART Training Algorithm

Scikit-Learn uses the `Classification & Regression Tree (CART)` algorithm to train DTrees (also called growing trees).

The algorithm works by first splitting the training set into two subsets using a single feature `k` and a theshold `t` (e.g petal length > 2.45cm). It chooses `t` and `k` via searching for the pair (`k, t`) that produces the purest subsets weighted by their size via the CART cost function.

Once the CART algorithm has successfully split the training set in two, it splits the subsets recursivly until the max_depth hyperparameter is reached.

## Regularization Hyperparameters

To avoid overfitting the training data, you need to restrict thee Dtree's freedom during training, which is called regularization. The regularization hyperparameters depend on the algorithm used, but generally, regularization can be controlled through the `max_depth` hyperparameter (the default value is `None`, which means unlimited depth). Reducing the `max_depth` will regularize the model and thus reduce the risk of overfitting.

The `DecisionTreeClassifier` class has a few other paremters that similarly restrict the shape of DTree:

   - `min_samples_split`: The minimum # of samples a node must have before it can be split
   - `min_samples_leaf`: The minimum # of samples a leaf node must have.
   - `min_weight_fraction_leaf`: Same as `min_samples_leaf` but as expressed as a fraction rather than a #.
   - `max_leaf_nodes`: The maximum # of leaf nodes.
   - `max_features`: The maximum # of features that are evaulated for splitting each node.

## Regression

Decision Treee's are also capable of performing regression tasks:

In [16]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(max_depth=2)
tree_reg.fit(X, y)

DecisionTreeRegressor(max_depth=2)

In [17]:
from sklearn.tree import export_graphviz

export_graphviz(
    tree_reg,
    out_file='iris_tree_reg.dot',
    feature_names=iris.feature_names[2:],
    class_names=iris.target_names,
    rounded=True,
    filled=True
)

In [18]:
!dot -Tpng iris_tree_reg.dot -o iris_tree_reg.png

![Decision Tree](iris_tree_reg.png)

The prediction in each leaf node is the average target value of the samples in that leaf node.

## Instability

The largest issue with Dtree's is that they are very sensative to small variations in the training data. For example, if you remove the widest `Iris Versicolor` from the iris traning set & train a new Dtree, you may get a model that looks very different from a model that has that data point included.

Random Forests can limit this instability averaging predictions over many trees as we will see in the next chapter.

## Exercises